In [ ]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%run Cplex_setup.ipynb
%run Data_Center_setup.ipynb

$$
\begin{equation*}
\begin{array}{lll}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{array}
\end{equation*}
$$

In [65]:
# first import the Model class from docplex.mp
from docplex.mp.model import Model

# create one model instance, with a name
DCM = Model(name='Data_center_demo')

# continuous decision variables
# charging and discharging of battery rate
p_char = {t: DCM.continuous_var(name='p_char_{0}'.format(t)) for t in range(T)}
p_dis = {t: DCM.continuous_var(name='p_dis_{0}'.format(t)) for t in range(T)}

# zone i AC temperature
Temp_AC = {(i,t): DCM.continuous_var(name='T_sup_{0}_{1}'.format(i,t)) for i in range(I_zone) for t in range(T)}

# ventilation air flow rate
v_vent = {t: DCM.continuous_var(name='v_vent_{0}'.format(t)) for t in range(T)}

# continuous state variables
# room air temperature
Temp_Zone = {(i,t): DCM.continuous_var(name='T_Zone_{0}_{1}'.format(i,t)) for i in range(I_zone) for t in range(T + 1)}

# battery state 
Battery_state = {(t): DCM.continuous_var(name='Battery_state_{0}'.format(t)) for t in range(T + 1)}

# eletricity demand
e_in = {t: DCM.continuous_var(name='e_in_{0}'.format(t)) for t in range(T)} 

# binary continuous variables
# charging and discharging of battery state
u_char = {t: DCM.binary_var(name='u_char_{0}'.format(t)) for t in range(T)}
u_dis = {t: DCM.binary_var(name='u_dis_{0}'.format(t)) for t in range(T)}

# chiller and tower on/off state
x_chiller = {(j,t): DCM.binary_var(name='x_chiller_{0}_{1}'.format(j,t)) for j in range(J_chiller) for t in range(T)}
x_tower = {(j,t): DCM.binary_var(name='x_tower_{0}_{1}'.format(j,t)) for j in range(J_tower) for t in range(T)}


In [66]:
# objective function
DCM.minimize(DCM.sum( e_in[t] * p_eg[t] for t in range(T)))

# constarint 1
for t in range(T):
    DCM.add(e_in[t] == e_server[t] + p_char[t] * eta_char_b - p_dis[t] * eta_dischar_b + e_solar[t] + e_sup[t] +\
                      beta_vent_0 + beta_vent_1 * (v_vent[t] - v_vent_startup)  +\
                      DCM.sum(x_chiller[j,t] * (beta_chiller_0[j] +  beta_chiller_1[j] * M_chiller_rated[j]) for j in range(J_chiller)) +\
                      DCM.sum(x_tower[j,t] * (beta_tower_0[j] +  beta_tower_1[j] * M_tower_rated[j]) for j in range(J_tower)) +\
                      beta_pump_0 + beta_pump_1 / (T_chiller_water_return[t] - T_chiller_water_supply[t]) * ((T_out[t] -  DCM.sum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * C_air +\
                      DCM.sum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * C_air))
    
# constarint 2
for t in range(T):
    DCM.add(e_in[t] >= 0)

# constarint 3
DCM.add(Battery_state[0] == 0.5 * Battery_capacity)
   
# constarint 3.5
for t in range(T):
    DCM.add(Battery_state[t + 1] == Battery_state[t] + p_char[t] * eta_char_b - p_dis[t] * eta_dischar_b)
    DCM.add(Battery_state[t + 1] >= Battery_capacity_lowbound)
    DCM.add(Battery_state[t + 1] <= Battery_capacity_upbound)
    
    
# constarint 4 & 5
for t in range(T):
    DCM.add(p_char[t] <= char_b_upperbound * u_char[t])
    DCM.add(p_dis[t] <= dis_b_upperbound * u_dis[t])

# constarint 6
for t in range(T):
    DCM.add(u_char[t] + u_dis[t] <= 1)
    
# constarint 7 & 7.5 
for i in range(I_zone):
    #print(i // I_zone_length, i % I_zone_length)
    Temp_Zone[i,0] == T_zone_initial[i// I_zone_length, i % I_zone_length]  # unit: C
for i in range(I_zone):
    for t in range(T):
        Temp_Zone[i, t + 1] =  DCM.sum(adjacency * (Temp_Zone[node,t] - Temp_Zone[i,t])  for node, adjacency in enumerate(adjacent_zone_map[i,:]))/(C_air * R_air_ij) + \
        (m_air_in[i] * c_s_air[i] * (Temp_AC[i,t] - Temp_Zone[i,t]) + theta_heat[i,t]) / (C_air) 

# constarint 8 & 8.5
for t in range(1, T + 1):
    for i in range(I_zone):
        DCM.add(Temp_Zone[i,t] >= T_datacenter_lowerbound[i,t-1])
        DCM.add(Temp_Zone[i,t] <= T_datacenter_upperbound[i,t-1])
        
        
# constarint 9 & 9.5
for t in range(T):
    DCM.add(v_vent[t] + v_out[t] >= v_vent_lowerbound[t])
    
# constarint 10
for t in range(T):
    DCM.add(DCM.sum( x_tower[j,t] * M_chiller_rated[j] for j in range(J_chiller)) >= \
    ((T_out[t] -  DCM.sum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * C_air + \
    DCM.sum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * C_air)/ \
    (T_chiller_water_return[t] - T_chiller_water_supply[t]))
    
# constarint 11 
for t in range(T):
    DCM.add(DCM.sum( x_tower[j,t] * M_tower_rated[j] for j in range(J_tower)) >= \
    ((T_out[t] -  DCM.sum(Chi[t] * Temp_Zone[i,t] for i in range(I_zone))) * v_out[t] * C_air + \
    DCM.sum(Chi[i] * (Temp_Zone[i,t] - Temp_AC[i,t]) for i in range(I_zone)) * (v_sup[t] - v_out[t]) * C_air)/ \
    (T_tower_water_return[t] - T_tower_water_supply[t]))
    
# constarint 12(added)
for i in range(I_zone):
    for t in range(T):
        DCM.add(Temp_AC[i,t] >= T_AC_lowerbound)
        DCM.add(Temp_AC[i,t] <= T_AC_upperbound)

    

DCM_sol = DCM.solve()
assert DCM_sol
DCM_sol.display()

solution for: Data_center_demo
objective: 181258.183
p_char_2 = 30.000
p_dis_0 = 50.000
p_dis_3 = 43.529
T_sup_0_0 = 30.000
T_sup_0_1 = 16.947
T_sup_0_2 = 16.947
T_sup_0_3 = 16.947
T_sup_1_0 = 30.000
T_sup_1_1 = 16.947
T_sup_1_2 = 16.947
T_sup_1_3 = 16.947
T_sup_2_0 = 30.000
T_sup_2_1 = 16.946
T_sup_2_2 = 16.946
T_sup_2_3 = 16.946
T_sup_3_0 = 30.000
T_sup_3_1 = 16.947
T_sup_3_2 = 16.947
T_sup_3_3 = 16.947
T_sup_4_0 = 30.000
T_sup_4_1 = 16.947
T_sup_4_2 = 16.947
T_sup_4_3 = 16.947
T_sup_5_0 = 30.000
T_sup_5_1 = 16.946
T_sup_5_2 = 16.946
T_sup_5_3 = 16.946
T_sup_6_0 = 30.000
T_sup_6_1 = 16.946
T_sup_6_2 = 16.946
T_sup_6_3 = 16.946
T_sup_7_0 = 30.000
T_sup_7_1 = 16.946
T_sup_7_2 = 16.946
T_sup_7_3 = 16.946
T_sup_8_0 = 30.000
T_sup_8_1 = 16.945
T_sup_8_2 = 16.945
T_sup_8_3 = 16.945
v_vent_0 = 0.300
v_vent_1 = 0.300
v_vent_2 = 0.300
v_vent_3 = 0.300
T_Zone_0_0 = 35.055
T_Zone_1_0 = 35.055
T_Zone_2_0 = 35.055
T_Zone_3_0 = 35.055
T_Zone_4_0 = 35.055
T_Zone_5_0 = 35.055
T_Zone_6_0 = 35.055
T_Z